In [1]:
import pandas as pd

In [2]:
data=pd.read_csv(r'C:\Users\user\Desktop\Varsha\processed_training.csv')

In [3]:
data.head()

,Major code stored as workover/drilling,Cde,Service hours type,Minor Repair / Tripping / Complication,Service hours type_lowercase
0,RDRP,1A,RIG CARRIER MOVED TO NEW SITE,NaN,rig carrier moved to new site
1,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job
2,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job
3,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job
4,OUC,24J,DWS UNDER MAJOR REPAIR & WAITG FOR COMP OF CIV...,NaN,dws under major repair waitg for comp of civ job


In [4]:
print(data['Cde'].value_counts())

Cde
6A     67930
5A     38754
2A     22671
23A    13612
11A    13301
       ...  
23I       76
7B        70
11B       56
24K       45
21J       10
Name: count, Length: 73, dtype: int64


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
threshold=250
minority_classes=data['Cde'].value_counts()[data['Cde'].value_counts()<threshold].index
minority_data = data[data['Cde'].isin(minority_classes)]

In [26]:
minority_data.shape

(2098, 5)

In [27]:
value_counts=minority_data['Cde'].value_counts()
minority_classes=value_counts[value_counts < threshold].index


In [28]:
minority_classes

Index(['21C', '24C', '24B', '24H', '23D', '21D', '1D', '21I', '2B', '23H',
       '1C', '21H', '23I', '7B', '11B', '24K', '21J'],
      dtype='object', name='Cde')

In [32]:
!pip install sentence-transformers


In [33]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load SBERT model for sentence embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

D:\hehe\envs\pytorch\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

D:\hehe\envs\pytorch\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

D:\hehe\envs\pytorch\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [37]:
top_k = 5
keywords_per_class = {}

for minority_class in minority_classes:
    # Extract data for the current minority class
    minority_class_data = data[data['Cde'] == minority_class]['Service hours type_lowercase'].tolist()
    
    # Generate document-level embedding for the current minority class
    doc_embeddings = model.encode(minority_class_data)
    Vmin = np.mean(doc_embeddings, axis=0)

    # Extract unique words from the minority class vocabulary
    vocab_min = set(word for text in minority_class_data for word in text.split())

    # Generate embeddings for each word in the vocabulary
    word_embeddings = {word: model.encode(word) for word in vocab_min}

    # Calculate cosine similarity between each word embedding and Vmin
    cos_sim_values = {word: cosine_similarity([embedding], [Vmin])[0][0] for word, embedding in word_embeddings.items()}

    # Sort words by cosine similarity and select top-k as keywords
    sorted_keywords = sorted(cos_sim_values.items(), key=lambda item: item[1], reverse=True)
    keywords = [word for word, similarity in sorted_keywords[:top_k]]

    # Store keywords for the current minority class
    keywords_per_class[minority_class] = keywords

# Output keywords for each minority class
for minority_class, keywords in keywords_per_class.items():
    print(f"Top-{top_k} keywords for {minority_class}: {keywords}")


Top-5 keywords for 21C: ['progress', 'readiness', 'civil', 'waiting', 'civilwks']
Top-5 keywords for 24C: ['logistics', 'shortage', 'fleet', 'availability', 'tanker']
Top-5 keywords for 24B: ['readiness', 'waiting', 'rig', 'standby', 'locationrig']
Top-5 keywords for 24H: ['rig', 'inspectionrepaired', 'weld', 'repair', 'commissioning']
Top-5 keywords for 23D: ['milling', 'millingwdntagged', 'millingmilled', 'millingobserved', '2865m']
Top-5 keywords for 21D: ['waiting', 'waited', 'progress', 'waitng', 'meantime']
Top-5 keywords for 1D: ['skidded', 'skided', 'skidedclearing', 'skid', 'skidding']
Top-5 keywords for 21I: ['waiting', 'waited', 'ctu', 'progress', 'vesselmtime']
Top-5 keywords for 2B: ['30conduct', '30csg', '30conductor', '30conductorx52', '30csgresumed']
Top-5 keywords for 23H: ['frac', 'frac5', 'jobpressure', 'fracjob', 'hydfrac']
Top-5 keywords for 1C: ['anchoring', 'rig', 'anchor4', 'anchor', 'anchor8chkd']
Top-5 keywords for 21H: ['cementng', 'cementation', 'cementing',